**Clone LLFF GitHub Repository (required)**

In [ ]:
!git clone https://github.com/Fyusion/LLFF.git

**Download pre-trained Checkpoint (required)**

In [ ]:
!wget https://www.dropbox.com/s/0pu4xsbrtrzytnt/llff_trained_model.zip?dl=1 -O llff_trained_model.zip
!unzip llff_trained_model.zip -d LLFF/checkpoints
!rm llff_trained_model.zip

**Download Test Scene (optional)**

In [ ]:
!wget http://people.eecs.berkeley.edu/~bmild/llff/data/testscene.zip
!unzip testscene.zip -d data
!rm testscene.zip

**Mount Google Drive and Copy LLFF_data (required)**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
#import shutil
#shutil.rmtree("/content/MyDrive")
!mkdir /content/MyDrive
!cp -R /content/gdrive/My\ Drive/LLFF_data /content/MyDrive/

**Install Required Packages (obviously required)**

In [ ]:
!apt-get update
!apt-get install -y colmap ffmpeg libglfw3-dev imagemagick libxcursor-dev libxinerama-dev
!pip install jupyterlab "tensorflow==1.14.0" "tensorflow-gpu==1.14.0" matplotlib scipy numpy scikit-image "imgaug==0.2.5" "imageio==2.4.0"

**Pose Estimation using COLMAP (CPU)**

In [ ]:
!colmap feature_extractor --database_path "/content/MyDrive/LLFF_data/in/database.db" --image_path "/content/MyDrive/LLFF_data/in/images" --ImageReader.single_camera 1 --SiftExtraction.use_gpu 0
!colmap exhaustive_matcher --database_path "/content/MyDrive/LLFF_data/in/database.db" --SiftMatching.use_gpu 0
!mkdir /content/MyDrive/LLFF_data/in/sparse
!colmap mapper --database_path "/content/MyDrive/LLFF_data/in/database.db" --image_path "/content/MyDrive/LLFF_data/in/images" --export_path "/content/MyDrive/LLFF_data/in/sparse" --Mapper.num_threads 16 --Mapper.init_min_tri_angle 4 --Mapper.multiple_models 0 --Mapper.extract_colors 0

%cd /content/LLFF
!python imgs2poses.py "/content/MyDrive/LLFF_data/in"
%cd ..

'''DEBUGGING STUFF (uncomment if you f.e. want to reuse the pose data in another session)'''
#!cp /content/MyDrive/LLFF_data/in/database.db /content/gdrive/My\ Drive/LLFF_data/in/database.db
#!cp /content/MyDrive/LLFF_data/in/poses_bounds.npy /content/gdrive/My\ Drive/LLFF_data/in/poses_bounds.npy
#!cp -R /content/MyDrive/LLFF_data/in/sparse /content/gdrive/My\ Drive/LLFF_data/in/

**Generate MPIs from Images and Pose Data (CPU)**

In [ ]:
%cd /content/LLFF
!python imgs2mpis.py "/content/MyDrive/LLFF_data/in" "/content/MyDrive/LLFF_data/out/mpis" --height 480
%cd ..
!cp -R /content/MyDrive/LLFF_data/out/mpis /content/gdrive/My\ Drive/LLFF_data/out/

**Build Cuda Renderer (optional, required for video rendering)**

In [ ]:
%cd /content/LLFF/cuda_renderer
!make
%cd ..
#!cp -R cuda_renderer /content/gdrive/My\ Drive/LLFF_data/

**Build OpenGL Viewer (optional)**

In [ ]:
%cd /content/LLFF/opengl_viewer
!make
%cd ..
!cp -R opengl_viewer /content/gdrive/My\ Drive/LLFF_data/

**Generate Testpath for Video (optional)**

In [ ]:
%cd /content/LLFF
!python imgs2renderpath.py "/content/MyDrive/LLFF_data/in" "/content/MyDrive/LLFF_data/out/renderpath.txt" --spiral
#!cp /content/MyDrive/LLFF_data/out/renderpath.txt /content/gdrive/My\ Drive/LLFF_data/out/renderpath.txt
%cd ..

**Render Video using Cuda Renderer (optional, requires GPU)**

In [ ]:
!/content/LLFF/cuda_renderer/cuda_renderer \
    /content/MyDrive/LLFF_data/out/mpis \
    /content/MyDrive/LLFF_data/out/renderpath.txt \
    /content/MyDrive/LLFF_data/out/video.mp4 \
    480 1 18
!cp /content/MyDrive/LLFF_data/out/video.mp4 /content/gdrive/My\ Drive/LLFF_data/out/video.mp4